# Define Variables

In [1]:
# Define the FOV of the camera
FOV = 160  # CSI Camera = 160 degrees

# KeyPoints TRT Model Path
OPTIMIZED_MODEL = 'Files/resnet18_baseline_att_224x224_A_epoch_249_trt.pth'

# Reset the camera

In [2]:
import getpass
import os

# Reset Camera
password = 'jetbot'
command = "sudo -S systemctl restart nvargus-daemon" #can be any command but don't forget -S as it enables input from stdin
os.system('echo %s | %s' % (password, command))

0

# Load pre-Trained keyPointsRCNN

In [3]:
from threading import Thread
from torch2trt import TRTModule
import trt_pose.models
import torch2trt
import os
import json
import trt_pose.coco
import trt_pose
import torch
import time

# Topology
with open('Files/human_pose.json', 'r') as f:
    human_pose = json.load(f)
topology = trt_pose.coco.coco_category_to_topology(human_pose)

# load Model
WIDTH = 224
HEIGHT = 224
data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()
model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))
print("KeyPoints RCNN Successfully Loaded. ")

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-fjxiq0ce because the default path (/home/jetbot/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Matplotlib is building the font cache; this may take a moment.


KeyPoints RCNN Successfully Loaded. 


# Prepare the Camera

In [4]:
import cv2
import torchvision.transforms as transforms
import PIL.Image
import time

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [5]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=15)
camera.running = True

# Prepare KeyPoints drawing on Frames

In [6]:
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

parse_objects = ParseObjects(topology)
draw_objects = DrawObjects(topology)

# Function to obtain keypoints (for drawing & calculating angle)

In [7]:
def get_keypoints(image, human_pose, topology, object_counts, objects, normalized_peaks):
    """Get the keypoints from torch data and put into a dictionary where keys are keypoints
    and values the x,y coordinates. The coordinates will be interpreted on the image given.

    Args:
        image: cv2 image
        human_pose: json formatted file about the keypoints

    Returns:
        dictionary: dictionary where keys are keypoints and values are the x,y coordinates
    """
    height = image.shape[0]
    width = image.shape[1]
    keypoints = {}
    K = topology.shape[0]
    count = int(object_counts[0])

    for i in range(count):
        obj = objects[0][i]
        C = obj.shape[0]
        for j in range(C):
            k = int(obj[j])
            if k >= 0:
                peak = normalized_peaks[0][j][k]
                x = round(float(peak[1]) * width)
                y = round(float(peak[0]) * height)
                keypoints[human_pose["keypoints"][j]] = (x, y)

    return keypoints

# Function to Calculate Angle

In [ ]:
def calculate_angle(WIDTH, keypoints, FOV=160):
    '''
    Arguments:
        + Image width
        + Dictionary of keypoints
        + Field of vision of camera (in degrees). Default = 160 if using CSI camera with fish-eye lens
        
    Returns:
        + Angle at which the object is located from respect to the center    
    '''

    keys = keypoints.values()
    
    # if at least 2 points are detected
    if len(keys) > 1:  
        object_location = int(sum([x for x,y in keys])/len(keys))
        angle = int(-FOV*(WIDTH/2 - object_location)/WIDTH)
        return [angle], keypoints
    
    
    else:
        return [100], keypoints

# Calculate Stage (optional)

In [ ]:
def calculate_stage(keypoints):
    '''
    Estimates the phase based on the skeleton parts detected:
    
    Phase 1: Knees and ankles are detected.
    Phase 2: Skeleton is detected but one of the knees or ankles is missing (obstructed by obstacle).
    Phase 3: No skeleton is detected.
    
    '''
    
    # If camera detects skeleton
    if keypoints:
        
        leg_parts = ["left_knee", "right_knee", "left_ankle", "right_ankle"] 
        obstructed = [x for x in leg_parts if x not in keypoints.keys()]
        
        # If legs are obstructed
        if obstructed:
            return [0,1,0]  # Phase 2
        
        else:
            return [1,0,0]   # Phase 1
            
    # No skeleton detected
    else:
        return [0,0,1]   # Phase 3

# Thread to print Angle & Stage & Lidar

In [ ]:
from IPython.display import clear_output as cls

def print_angle_thread():
    
    global print_angle_running
    global global_angle
    print_angle_running = True
    global visualize_stage
    global visualize_lidar
    global angles
    global previous_readings
    
    
    while True:
        keypoints, image, counts, objects, peaks = execute2({'new': camera.value})
        angle, _ = calculate_angle(WIDTH, keypoints, FOV)
        
        # If there was a large angle difference, update
        if global_angle:
            if abs(global_angle-angle[0])>4:
                global_angle = angle[0]
        
        # If first time running, define global_angle
        else:
            global_angle = angle[0]   
            
        # If you want to print ANGLE, LiDAR & STAGE
        if visualize_stage and visualize_lidar:
            stage = calculate_stage(keypoints)
            previous_readings = read_lidar_wrapper(angles, previous_readings)
            
            out.value = f"<pref><font color='black'><span style='background-color:#A9CDF8'>  1. Angle:</span><font color='blue'> {global_angle}<br>\
                                <font color='black'><span style='background-color:#E1A3A3'>  2. Stage:</span><font color='red'> {stage}<br>\
                                <font color='black'><span style='background-color:#CFBBE8'>  3. LiDAR:</span><font color='purple'> {previous_readings}</pref>"
            
        # If you want to print ANGLE & STAGE
        elif visualize_stage and not visualize_lidar:
            stage = calculate_stage(keypoints)
            out.value = f"<pref>Angle: {global_angle}\nStage: {stage}</pref>"
        
        # If you want to print ANGLE & LIDAR
        elif visualize_lidar and not visualize_stage:
            previous_readings = read_lidar_wrapper(angles, previous_readings)
            out.value = f"<pref>Angle: {global_angle}\nLiDAR: {previous_readings}</pref>"
            
        # IF you want to ONLY print ANGLE
        else:
            out.value = f"<pref>Angle: {global_angle}</pref>"


# Define Execute (For Visualization)

In [ ]:
def execute(change):
    
    global image
    global thread2
    global keypoints

    image = change['new']
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)#, cmap_threshold=0.15, link_threshold=0.15)
    keypoints = get_keypoints(image, human_pose, topology, counts, objects, peaks)

    if print_angle_running == False:
        thread2 = Thread(target = print_angle_thread)
        thread2.daemon = True
        thread2.start()
        
    draw_objects(image, counts, objects, peaks)
    image_w.value = bgr8_to_jpeg(image[:, ::-1, :])

# Define Execute (Used for angle calculation)

In [ ]:
def execute2(change, visualize=True):
    image = change['new']
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)#, cmap_threshold=0.15, link_threshold=0.15)
    keypoints = get_keypoints(image, human_pose, topology, counts, objects, peaks)

    return keypoints, image, counts, objects, peaks

###### 2 Different execute functions are defined because if only one is used and the variables are made global to be shared by both threads, the delay goes from 2 seconds to 20~30 seconds 

In [ ]:
print_angle_running = False  # Thread Lock (prevents from crashes)
global global_angle
global_angle = False

# Run Everything

In [ ]:
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

#### Uncomment Below if you want to run this independently

In [ ]:
# image_w = ipywidgets.Image(format='jpeg', width=WIDTH, height=HEIGHT)
# display(image_w)

In [ ]:
# execute({'new': camera.value})
# camera.observe(execute, names='value')